This script is for converting the matsim output files (generated by Aurore) into street graphs

In [1]:
import copy
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import geopandas as gpd
import shapely as shp
import pyproj
import numpy as np

import snman
from snman.constants import *
from snman import osmnx_customized as oxc

PERIMETER = '_accessibility_debug'

# Set these paths according to your own setup
data_directory = os.path.join(
    'C:',os.sep,'Users','lballo','polybox','Research',
    'SNMan','SNMan Shared','data_v2'
)
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)
outputs_path = os.path.join(data_directory, 'outputs', PERIMETER)
paper_path = os.path.join(
    'C:',os.sep,'Users','lballo','polybox','Research',
    'E-Bike City Accessibility','EBC Accessibility Paper - Shared'
)

#matsim_results_path = os.path.join(
#    paper_path, 'MATSim results', '2024-07-12 Travel times before and after'
#)

matsim_results_path = os.path.join(
    paper_path, 'MATSim results', '2024-07-17 with link counts'
)

#CRS_internal = 29119    # for Boston
#CRS_internal = 32216    # for Chicago
CRS_internal = 2056      # for Zurich
CRS_for_export = 4326
oxc.settings.useful_tags_way = OSM_TAGS

In [2]:
TIME = '18:00'
STATE = 'after'

In [3]:
# read the original osm export
osm_export = snman.io.import_geofile_to_gdf(
    os.path.join(matsim_results_path, f'{STATE}_oneway_links_exploded.gzip')
)

In [4]:
# read the matsim output
tt = pd.read_csv(
    os.path.join(matsim_results_path, f'{STATE}_bike100pct.csv')
)
tt.set_index('OSM_ID', inplace=True)
tt

,LinkId,FreeflowTTCar,FreeFlowTTBike,car_0:00,N_cars_0:00,bike_0:00,N_bikes_0:00,car_0:30,N_cars_0:30,bike_0:30,...,N_bikes_22:30,car_23:00,N_cars_23:00,bike_23:00,N_bikes_23:00,car_23:30,N_cars_23:30,bike_23:30,N_bikes_23:30,Unnamed: 196
OSM_ID,,,,,,,,,,,,,,,,,,,,,
7701054,3640,22.34,44.68,NaN,0,NaN,0,NaN,0,NaN,...,2,29.00,19,NaN,0,24.62,13,NaN,0,NaN
7701052,3638,13.91,27.81,NaN,0,NaN,0,NaN,0,NaN,...,0,13.67,3,NaN,0,20.00,1,NaN,0,NaN
7701053,3639,9.81,19.62,NaN,0,NaN,0,NaN,0,NaN,...,0,16.00,11,NaN,0,16.00,3,NaN,0,NaN
7715687,3630,5.86,11.73,NaN,0,NaN,0,12.0,1,NaN,...,2,12.00,127,15.0,3,12.00,98,15.0,3,NaN
7715688,3631,4.39,8.79,NaN,0,NaN,0,NaN,0,NaN,...,0,11.00,3,12.0,1,11.00,2,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7715686,3629,9.62,19.24,NaN,0,NaN,0,NaN,0,NaN,...,1,9.00,15,23.0,3,16.00,5,12.0,2,NaN
7591240,75829,3.14,10.48,NaN,0,NaN,0,NaN,0,NaN,...,0,10.00,5,14.0,1,10.00,4,NaN,0,NaN
7701043,3628,17.95,35.91,NaN,0,NaN,0,NaN,0,NaN,...,0,NaN,0,NaN,0,18.00,1,NaN,0,NaN


In [5]:
# join the matsim output with the input file
osm_export['osm_id'] = osm_export['osm_id'].astype('int64')

m = pd.merge(
    tt.reset_index(), osm_export[['osm_id', 'highway', 'geometry']],
    left_on='OSM_ID', right_on='osm_id', how='left'
)
m.drop(columns='osm_id', inplace=True)
m = gpd.GeoDataFrame(m)
m

,OSM_ID,LinkId,FreeflowTTCar,FreeFlowTTBike,car_0:00,N_cars_0:00,bike_0:00,N_bikes_0:00,car_0:30,N_cars_0:30,...,N_cars_23:00,bike_23:00,N_bikes_23:00,car_23:30,N_cars_23:30,bike_23:30,N_bikes_23:30,Unnamed: 196,highway,geometry
0,7701054,3640,22.34,44.68,NaN,0,NaN,0,NaN,0,...,19,NaN,0,24.62,13,NaN,0,NaN,residential,"LINESTRING (2701971.250 1264550.448, 2702085.5..."
1,7701052,3638,13.91,27.81,NaN,0,NaN,0,NaN,0,...,3,NaN,0,20.00,1,NaN,0,NaN,residential,"LINESTRING (2701971.250 1264550.448, 2701866.5..."
2,7701053,3639,9.81,19.62,NaN,0,NaN,0,NaN,0,...,11,NaN,0,16.00,3,NaN,0,NaN,residential,"LINESTRING (2701971.250 1264550.448, 2701945.5..."
3,7715687,3630,5.86,11.73,NaN,0,NaN,0,12.0,1,...,127,15.0,3,12.00,98,15.0,3,NaN,secondary,"LINESTRING (2699643.283 1265696.180, 2699595.2..."
4,7715688,3631,4.39,8.79,NaN,0,NaN,0,NaN,0,...,3,12.0,1,11.00,2,NaN,0,NaN,residential,"LINESTRING (2699596.957 1265741.904, 2699595.2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143330,7715686,3629,9.62,19.24,NaN,0,NaN,0,NaN,0,...,15,23.0,3,16.00,5,12.0,2,NaN,secondary,"LINESTRING (2699643.283 1265696.180, 2699722.1..."
143331,7591240,75829,3.14,10.48,NaN,0,NaN,0,NaN,0,...,5,14.0,1,10.00,4,NaN,0,NaN,tertiary,"LINESTRING (2699666.473 1236986.332, 2699708.9..."
143332,7701043,3628,17.95,35.91,NaN,0,NaN,0,NaN,0,...,0,NaN,0,18.00,1,NaN,0,NaN,residential,"LINESTRING (2702296.218 1264387.280, 2702354.5..."
143333,7701042,3627,11.98,23.97,NaN,0,NaN,0,NaN,0,...,0,NaN,0,NaN,0,NaN,0,NaN,residential,"LINESTRING (2702296.218 1264387.280, 2702344.6..."


In [6]:
# create start and end points from line geometries
m['u'] = m.apply(
    lambda row: shp.Point(row['geometry'].coords[0]),
    axis=1
)
m['v'] = m.apply(
    lambda row: shp.Point(row['geometry'].coords[-1]),
    axis=1
)
m

,OSM_ID,LinkId,FreeflowTTCar,FreeFlowTTBike,car_0:00,N_cars_0:00,bike_0:00,N_bikes_0:00,car_0:30,N_cars_0:30,...,N_bikes_23:00,car_23:30,N_cars_23:30,bike_23:30,N_bikes_23:30,Unnamed: 196,highway,geometry,u,v
0,7701054,3640,22.34,44.68,NaN,0,NaN,0,NaN,0,...,0,24.62,13,NaN,0,NaN,residential,"LINESTRING (2701971.250 1264550.448, 2702085.5...",POINT (2701971.250 1264550.448),POINT (2702126.195 1264452.318)
1,7701052,3638,13.91,27.81,NaN,0,NaN,0,NaN,0,...,0,20.00,1,NaN,0,NaN,residential,"LINESTRING (2701971.250 1264550.448, 2701866.5...",POINT (2701971.250 1264550.448),POINT (2701866.521 1264600.062)
2,7701053,3639,9.81,19.62,NaN,0,NaN,0,NaN,0,...,0,16.00,3,NaN,0,NaN,residential,"LINESTRING (2701971.250 1264550.448, 2701945.5...",POINT (2701971.250 1264550.448),POINT (2701945.593 1264472.830)
3,7715687,3630,5.86,11.73,NaN,0,NaN,0,12.0,1,...,3,12.00,98,15.0,3,NaN,secondary,"LINESTRING (2699643.283 1265696.180, 2699595.2...",POINT (2699643.283 1265696.180),POINT (2699595.284 1265705.323)
4,7715688,3631,4.39,8.79,NaN,0,NaN,0,NaN,0,...,1,11.00,2,NaN,0,NaN,residential,"LINESTRING (2699596.957 1265741.904, 2699595.2...",POINT (2699596.957 1265741.904),POINT (2699595.284 1265705.323)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143330,7715686,3629,9.62,19.24,NaN,0,NaN,0,NaN,0,...,3,16.00,5,12.0,2,NaN,secondary,"LINESTRING (2699643.283 1265696.180, 2699722.1...",POINT (2699643.283 1265696.180),POINT (2699722.150 1265681.709)
143331,7591240,75829,3.14,10.48,NaN,0,NaN,0,NaN,0,...,1,10.00,4,NaN,0,NaN,tertiary,"LINESTRING (2699666.473 1236986.332, 2699708.9...",POINT (2699666.473 1236986.332),POINT (2699708.939 1236996.464)
143332,7701043,3628,17.95,35.91,NaN,0,NaN,0,NaN,0,...,0,18.00,1,NaN,0,NaN,residential,"LINESTRING (2702296.218 1264387.280, 2702354.5...",POINT (2702296.218 1264387.280),POINT (2702405.547 1264475.179)
143333,7701042,3627,11.98,23.97,NaN,0,NaN,0,NaN,0,...,0,NaN,0,NaN,0,NaN,residential,"LINESTRING (2702296.218 1264387.280, 2702344.6...",POINT (2702296.218 1264387.280),POINT (2702344.675 1264299.956)


In [7]:
# reconstruct nodes and give them new IDs
nd = pd.concat([m['u'], m['v']]).reset_index().rename(columns={0: 'geometry'})
nd['osmid'] = pd.factorize(nd['geometry'])[0]
nd[['x', 'y']] = nd.apply(
    lambda row: (row['geometry'].x, row['geometry'].y),
    axis=1,
    result_type='expand'
)
nd.drop(columns=['index'], inplace=True)
#nd.drop(columns='OSM_ID', inplace=True)
nd.drop_duplicates(inplace=True)
nd.set_index('osmid', inplace=True)
nd = gpd.GeoDataFrame(nd, geometry='geometry', crs=2056)

nd

,geometry,x,y
osmid,,,
0,POINT (2701971.250 1264550.448),2.701971e+06,1.264550e+06
1,POINT (2699643.283 1265696.180),2.699643e+06,1.265696e+06
2,POINT (2699596.957 1265741.904),2.699597e+06,1.265742e+06
3,POINT (2702126.195 1264452.318),2.702126e+06,1.264452e+06
4,POINT (2638563.748 1242835.818),2.638564e+06,1.242836e+06
...,...,...,...
55704,POINT (2675533.611 1236266.763),2.675534e+06,1.236267e+06
55705,POINT (2717068.782 1226934.599),2.717069e+06,1.226935e+06
55706,POINT (2647917.826 1249975.767),2.647918e+06,1.249976e+06


In [8]:
# write the new node IDs into the edge table and create an index like in the street graph

m2 = pd.merge(
    m.reset_index(), nd.reset_index()[['osmid', 'geometry']],
    how='left', left_on='u', right_on='geometry', suffixes=['', '_right']
).drop(columns=['u', 'geometry_right']).rename(columns={'osmid': 'u'})

m2 = pd.merge(
    m2, nd.reset_index()[['osmid', 'geometry']],
    how='left', left_on='v', right_on='geometry', suffixes=['', '_right']
).drop(columns=['v', 'geometry_right']).rename(columns={'osmid': 'v'})

m2['uv'] = m2.apply(lambda row: (row['u'], row['v']), axis=1)
m2['length'] = m2.apply(lambda row: row.geometry.length, axis=1)

m2

,index,OSM_ID,LinkId,FreeflowTTCar,FreeFlowTTBike,car_0:00,N_cars_0:00,bike_0:00,N_bikes_0:00,car_0:30,...,N_cars_23:30,bike_23:30,N_bikes_23:30,Unnamed: 196,highway,geometry,u,v,uv,length
0,0,7701054,3640,22.34,44.68,NaN,0,NaN,0,NaN,...,13,NaN,0,NaN,residential,"LINESTRING (2701971.250 1264550.448, 2702085.5...",0,3,"(0, 3)",186.182650
1,1,7701052,3638,13.91,27.81,NaN,0,NaN,0,NaN,...,1,NaN,0,NaN,residential,"LINESTRING (2701971.250 1264550.448, 2701866.5...",0,5858,"(0, 5858)",115.886530
2,2,7701053,3639,9.81,19.62,NaN,0,NaN,0,NaN,...,3,NaN,0,NaN,residential,"LINESTRING (2701971.250 1264550.448, 2701945.5...",0,3636,"(0, 3636)",81.749086
3,3,7715687,3630,5.86,11.73,NaN,0,NaN,0,12.0,...,98,15.0,3,NaN,secondary,"LINESTRING (2699643.283 1265696.180, 2699595.2...",1,51236,"(1, 51236)",48.861403
4,4,7715688,3631,4.39,8.79,NaN,0,NaN,0,NaN,...,2,NaN,0,NaN,residential,"LINESTRING (2699596.957 1265741.904, 2699595.2...",2,51236,"(2, 51236)",36.619106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143330,143330,7715686,3629,9.62,19.24,NaN,0,NaN,0,NaN,...,5,12.0,2,NaN,secondary,"LINESTRING (2699643.283 1265696.180, 2699722.1...",1,10144,"(1, 10144)",80.184124
143331,143331,7591240,75829,3.14,10.48,NaN,0,NaN,0,NaN,...,4,NaN,0,NaN,tertiary,"LINESTRING (2699666.473 1236986.332, 2699708.9...",55238,34380,"(55238, 34380)",43.657291
143332,143332,7701043,3628,17.95,35.91,NaN,0,NaN,0,NaN,...,1,NaN,0,NaN,residential,"LINESTRING (2702296.218 1264387.280, 2702354.5...",5,26196,"(5, 26196)",149.603609
143333,143333,7701042,3627,11.98,23.97,NaN,0,NaN,0,NaN,...,0,NaN,0,NaN,residential,"LINESTRING (2702296.218 1264387.280, 2702344.6...",5,32722,"(5, 32722)",99.867104


In [9]:
# replace "none" and "nan" travel times with free flow
# in case of paths, set car travel time to inf
import copy
m3 = copy.deepcopy(m2)

# fix a typo in the one column name
m3.rename(columns={'FreeflowTTCar': 'FreeFlowTTCar'}, inplace=True)

# build the 96 column names automatically
for mode in ['car', 'bike']:
    for hour in map(str, range(24)):
        for minute in ['00', '30']:
            column = f'{mode}_{hour}:{minute}'
            print(column)
            ff_tt_column = f'FreeFlowTT{mode.capitalize()}'
            # replace all nan values with the corresponding free flow travel time
            m3[column] = m3.apply(
                # cars on paths -> inf travel time
                lambda row: np.inf if row['highway'] == 'path' and mode == 'car'
                # nan -> free flow travel time
                else row[ff_tt_column] if np.isnan(row[column])
                # else -> no change
                else row[column],
                axis=1
            )
m3

car_0:00
car_0:30
car_1:00
car_1:30
car_2:00
car_2:30
car_3:00
car_3:30
car_4:00
car_4:30
car_5:00
car_5:30
car_6:00
car_6:30
car_7:00
car_7:30
car_8:00
car_8:30
car_9:00
car_9:30
car_10:00
car_10:30
car_11:00
car_11:30
car_12:00
car_12:30
car_13:00
car_13:30
car_14:00
car_14:30
car_15:00
car_15:30
car_16:00
car_16:30
car_17:00
car_17:30
car_18:00
car_18:30
car_19:00
car_19:30
car_20:00
car_20:30
car_21:00
car_21:30
car_22:00
car_22:30
car_23:00
car_23:30
bike_0:00
bike_0:30
bike_1:00
bike_1:30
bike_2:00
bike_2:30
bike_3:00
bike_3:30
bike_4:00
bike_4:30
bike_5:00
bike_5:30
bike_6:00
bike_6:30
bike_7:00
bike_7:30
bike_8:00
bike_8:30
bike_9:00
bike_9:30
bike_10:00
bike_10:30
bike_11:00
bike_11:30
bike_12:00
bike_12:30
bike_13:00
bike_13:30
bike_14:00
bike_14:30
bike_15:00
bike_15:30
bike_16:00
bike_16:30
bike_17:00
bike_17:30
bike_18:00
bike_18:30
bike_19:00
bike_19:30
bike_20:00
bike_20:30
bike_21:00
bike_21:30
bike_22:00
bike_22:30
bike_23:00
bike_23:30


,index,OSM_ID,LinkId,FreeFlowTTCar,FreeFlowTTBike,car_0:00,N_cars_0:00,bike_0:00,N_bikes_0:00,car_0:30,...,N_cars_23:30,bike_23:30,N_bikes_23:30,Unnamed: 196,highway,geometry,u,v,uv,length
0,0,7701054,3640,22.34,44.68,22.34,0,44.68,0,22.34,...,13,44.68,0,NaN,residential,"LINESTRING (2701971.250 1264550.448, 2702085.5...",0,3,"(0, 3)",186.182650
1,1,7701052,3638,13.91,27.81,13.91,0,27.81,0,13.91,...,1,27.81,0,NaN,residential,"LINESTRING (2701971.250 1264550.448, 2701866.5...",0,5858,"(0, 5858)",115.886530
2,2,7701053,3639,9.81,19.62,9.81,0,19.62,0,9.81,...,3,19.62,0,NaN,residential,"LINESTRING (2701971.250 1264550.448, 2701945.5...",0,3636,"(0, 3636)",81.749086
3,3,7715687,3630,5.86,11.73,5.86,0,11.73,0,12.00,...,98,15.00,3,NaN,secondary,"LINESTRING (2699643.283 1265696.180, 2699595.2...",1,51236,"(1, 51236)",48.861403
4,4,7715688,3631,4.39,8.79,4.39,0,8.79,0,4.39,...,2,8.79,0,NaN,residential,"LINESTRING (2699596.957 1265741.904, 2699595.2...",2,51236,"(2, 51236)",36.619106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143330,143330,7715686,3629,9.62,19.24,9.62,0,19.24,0,9.62,...,5,12.00,2,NaN,secondary,"LINESTRING (2699643.283 1265696.180, 2699722.1...",1,10144,"(1, 10144)",80.184124
143331,143331,7591240,75829,3.14,10.48,3.14,0,10.48,0,3.14,...,4,10.48,0,NaN,tertiary,"LINESTRING (2699666.473 1236986.332, 2699708.9...",55238,34380,"(55238, 34380)",43.657291
143332,143332,7701043,3628,17.95,35.91,17.95,0,35.91,0,17.95,...,1,35.91,0,NaN,residential,"LINESTRING (2702296.218 1264387.280, 2702354.5...",5,26196,"(5, 26196)",149.603609
143333,143333,7701042,3627,11.98,23.97,11.98,0,23.97,0,11.98,...,0,23.97,0,NaN,residential,"LINESTRING (2702296.218 1264387.280, 2702344.6...",5,32722,"(5, 32722)",99.867104


In [10]:
# write the chosen travel time window as master travel time
m3[f'cost_lanes_{MODE_PRIVATE_CARS}_{DIRECTION_FORWARD}'] = m3[f'car_{TIME}']
m3[f'cost_lanes_{MODE_PRIVATE_CARS}_{DIRECTION_BACKWARD}'] = np.inf

# set some attributes
m3['oneway'] = 'yes'

# add a simple single motorized lane in the forward direction,
# please note that this street graph is only needed to represent car travel times
# so the exact number of lanes does not matter
m3['lanes'] = m3.apply(
    lambda row: snman.space_allocation.SpaceAllocation(
        [snman.space_allocation.Lane(LANETYPE_MOTORIZED, DIRECTION_FORWARD)]
    ) if row['highway'] != 'path'
    else snman.space_allocation.SpaceAllocation(
        [snman.space_allocation.Lane(LANETYPE_FOOT, DIRECTION_FORWARD)]
    ),
    axis=1
)

m3.set_index('uv', inplace=True)
m3

,index,OSM_ID,LinkId,FreeFlowTTCar,FreeFlowTTBike,car_0:00,N_cars_0:00,bike_0:00,N_bikes_0:00,car_0:30,...,Unnamed: 196,highway,geometry,u,v,length,cost_lanes_private_cars_>,cost_lanes_private_cars_<,oneway,lanes
uv,,,,,,,,,,,,,,,,,,,,,
"(0, 3)",0,7701054,3640,22.34,44.68,22.34,0,44.68,0,22.34,...,NaN,residential,"LINESTRING (2701971.250 1264550.448, 2702085.5...",0,3,186.182650,28.46,inf,yes,[M>*3.0]
"(0, 5858)",1,7701052,3638,13.91,27.81,13.91,0,27.81,0,13.91,...,NaN,residential,"LINESTRING (2701971.250 1264550.448, 2701866.5...",0,5858,115.886530,20.32,inf,yes,[M>*3.0]
"(0, 3636)",2,7701053,3639,9.81,19.62,9.81,0,19.62,0,9.81,...,NaN,residential,"LINESTRING (2701971.250 1264550.448, 2701945.5...",0,3636,81.749086,16.06,inf,yes,[M>*3.0]
"(1, 51236)",3,7715687,3630,5.86,11.73,5.86,0,11.73,0,12.00,...,NaN,secondary,"LINESTRING (2699643.283 1265696.180, 2699595.2...",1,51236,48.861403,12.02,inf,yes,[M>*3.0]
"(2, 51236)",4,7715688,3631,4.39,8.79,4.39,0,8.79,0,4.39,...,NaN,residential,"LINESTRING (2699596.957 1265741.904, 2699595.2...",2,51236,36.619106,11.00,inf,yes,[M>*3.0]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(1, 10144)",143330,7715686,3629,9.62,19.24,9.62,0,19.24,0,9.62,...,NaN,secondary,"LINESTRING (2699643.283 1265696.180, 2699722.1...",1,10144,80.184124,15.59,inf,yes,[M>*3.0]
"(55238, 34380)",143331,7591240,75829,3.14,10.48,3.14,0,10.48,0,3.14,...,NaN,tertiary,"LINESTRING (2699666.473 1236986.332, 2699708.9...",55238,34380,43.657291,10.02,inf,yes,[M>*3.0]
"(5, 26196)",143332,7701043,3628,17.95,35.91,17.95,0,35.91,0,17.95,...,NaN,residential,"LINESTRING (2702296.218 1264387.280, 2702354.5...",5,26196,149.603609,18.00,inf,yes,[M>*3.0]


In [11]:
m4 = m3[[
    'OSM_ID', 'FreeFlowTTCar', 'car_7:00', 'car_18:00', 'N_cars_7:00', 'N_cars_18:00', 'highway', 'length', 'lanes', 'oneway', 'geometry'
]]

In [12]:
# create a new street graph
G = snman.street_graph.street_graph_from_gdf(nd, m3)

In [13]:
# save the street graph

if 1:
    snman.io.export_street_graph(
        G,
        os.path.join(outputs_path, f'tt_{STATE}_edges_all_fields.gpkg'),
        os.path.join(outputs_path, f'tt_{STATE}_nodes_all_fields.gpkg'),
        crs=CRS_for_export,
        stringify_additional_attributes=['lanes']
    )

In [14]:
STATE

'after'

In [15]:
if 0:
    H = snman.street_graph.street_graph_from_gdf(nd, m4)

In [16]:
if 0:
    snman.io.export_street_graph(
        H,
        os.path.join(outputs_path, f'tt_{STATE}_edges.gpkg'),
        os.path.join(outputs_path, f'tt_{STATE}_nodes.gpkg'),
        crs=CRS_for_export,
        stringify_additional_attributes=['lanes']
    )